In [15]:
import pdfplumber
import re
import unidecode
import os

pdf_folder = r"D:\Ciencia de Dados\Projeto Integrador de Extenção 3\RAGzilla\project\data\raw"

def clean_text(text):
    """Limpa o texto extraído do PDF."""
    if not text:
        return ""
    
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'__', '', text)
    return text.strip()

def extract_entities(text):
    """Extrai CNPJs, CPFs, RGs e CEPs do texto."""
    cnpjs = re.findall(r'\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}', text)
    cpfs  = re.findall(r'\d{3}\.\d{3}\.\d{3}-\d{2}', text)
    rgs   = re.findall(r'\d{1,2}\.\d{3}\.\d{3}-\d{1}', text)
    ceps  = re.findall(r'\d{5}-\d{3}', text)
    return cnpjs + cpfs + rgs + ceps

all_chunks = []
global_chunk_number = 0
max_tokens = 400
overlap = 30

import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")

def text_to_tokens(text):
    return encoding.encode(text)

def tokens_to_text(tokens):
    return encoding.decode(tokens)

def chunk_text_by_tokens(text, max_tokens, overlap):
    tokens = text_to_tokens(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk_tokens = tokens[start:end]
        chunk_text = tokens_to_text(chunk_tokens)
        # Ignora chunks menores que o overlap, exceto se tiver entidades
        if len(chunk_tokens) >= overlap or extract_entities(chunk_text):
            chunks.append(chunk_text)
        start += max_tokens - overlap
    return chunks

# Processa PDFs
for pdf_file in os.listdir(pdf_folder):
    if not pdf_file.lower().endswith(".pdf"):
        continue
    pdf_path = os.path.join(pdf_folder, pdf_file)
    
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                page_text = clean_text(page_text)
                chunks = chunk_text_by_tokens(page_text, max_tokens=max_tokens, overlap=overlap)
                
                for chunk in chunks:
                    # chunk original
                    all_chunks.append({
                        "chunk_number": global_chunk_number,
                        "text": chunk,
                        "source_file": pdf_file,
                        "page_number": i
                    })
                    global_chunk_number += 1
                    
                    # chunk só com entidades (mesmo que seja menor que overlap)
                    entities = extract_entities(chunk)
                    if entities:
                        entity_text = " ".join(entities)
                        all_chunks.append({
                            "chunk_number": global_chunk_number,
                            "text": entity_text,
                            "source_file": pdf_file,
                            "page_number": i
                        })
                        global_chunk_number += 1

print(f"Total de chunks gerados: {len(all_chunks)}")
print("Exemplo de chunk com metadados:\n", all_chunks[0])


Total de chunks gerados: 68
Exemplo de chunk com metadados:
 {'chunk_number': 0, 'text': 'CONTRATO DE ASSOCIAÇÃO DE EMPRESA Pelo presente instrumento e na melhor forma de direito, de um lado, AJJ SERVIÇOS CONTABÉIS LTDA, pessoa jurídica de direito privado, inscrita no CNPJ sob o nº 28. 913.706/0001-63, com sede na Rua Alfred Charvet, nº 710, Bairro Vila Nova, Araucária/PR, CEP: 83.703-278, neste ato, representada por seu sócio administrador, Jhonny Cézar de Jesus Falavinha, brasileiro, portador do RG nº 6.223.402-4 SSP/PR, inscrito no CPF nº 007.724.809-07, doravante denominada EMPRESA ASSOCIADA, e de outro lado, o PARQUE CIENTÍFICO E TECNOLÓGICO DE BIOCIÊNCIAS LTDA, pessoa jurídica de direito privado, inscrito no CNPJ sob o nº 21.526.709/0001-03, com sede na Rodovia PR-182, Km 320/321, s/n, Condomínio Industrial Biopark, no município de Toledo, Estado do Paraná, CEP 85.919-899, neste ato representada por seu sócio administrador, Victor Donaduzzi, brasileiro, inscrito no CPF sob o nº 0

In [16]:
import json
import os

# Pasta de destino
output_folder = r"D:\Ciencia de Dados\Projeto Integrador de Extenção 3\RAGzilla\project\data\processed"
os.makedirs(output_folder, exist_ok=True)

# Nome do arquivo de saída
output_file = os.path.join(output_folder, "chunks.json")

# Salvar todos os chunks
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=4)

print(f"✅ {len(all_chunks)} chunks salvos em {output_file}")

✅ 68 chunks salvos em D:\Ciencia de Dados\Projeto Integrador de Extenção 3\RAGzilla\project\data\processed\chunks.json


In [17]:
import json

input_file = r"D:\Ciencia de Dados\Projeto Integrador de Extenção 3\RAGzilla\project\data\processed\chunks.json"

with open(input_file, "r", encoding="utf-8") as f:
    chunks = json.load(f)

print(f"Total de chunks carregados: {len(chunks)}")
print("Exemplo:", chunks[0])

Total de chunks carregados: 68
Exemplo: {'chunk_number': 0, 'text': 'CONTRATO DE ASSOCIAÇÃO DE EMPRESA Pelo presente instrumento e na melhor forma de direito, de um lado, AJJ SERVIÇOS CONTABÉIS LTDA, pessoa jurídica de direito privado, inscrita no CNPJ sob o nº 28. 913.706/0001-63, com sede na Rua Alfred Charvet, nº 710, Bairro Vila Nova, Araucária/PR, CEP: 83.703-278, neste ato, representada por seu sócio administrador, Jhonny Cézar de Jesus Falavinha, brasileiro, portador do RG nº 6.223.402-4 SSP/PR, inscrito no CPF nº 007.724.809-07, doravante denominada EMPRESA ASSOCIADA, e de outro lado, o PARQUE CIENTÍFICO E TECNOLÓGICO DE BIOCIÊNCIAS LTDA, pessoa jurídica de direito privado, inscrito no CNPJ sob o nº 21.526.709/0001-03, com sede na Rodovia PR-182, Km 320/321, s/n, Condomínio Industrial Biopark, no município de Toledo, Estado do Paraná, CEP 85.919-899, neste ato representada por seu sócio administrador, Victor Donaduzzi, brasileiro, inscrito no CPF sob o nº 047.139.439-40, dorava

In [ ]:
from openai import OpenAI

from dotenv import load_dotenv
import os

# Carregar variáveis do .env
load_dotenv()

# OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

embeddings_list = []

for chunk in chunks:
    response = client.embeddings.create(
        model="text-embedding-3-small",  # text-embedding-3-large ; text-embedding-3-small
        input=chunk["text"]
    )
    emb = response.data[0].embedding
    embeddings_list.append({
        "chunk_number": chunk["chunk_number"],
        "embedding": emb,
        "source_file": chunk["source_file"],
        "page_number": chunk["page_number"]
    })

print("✅ Embeddings gerados para todos os chunks!")

✅ Embeddings gerados para todos os chunks!


In [19]:
print(embeddings_list[0:10])

[{'chunk_number': 0, 'embedding': [-0.0058996183797717094, -0.005506758112460375, -0.006762567441910505, 0.01993177831172943, -0.004250948783010244, -0.026486163958907127, 0.010704600252211094, 0.021892720833420753, -0.00026253637042827904, 0.05412740260362625, 0.015526372008025646, -0.000535985513124615, -0.009448790922760963, -0.06269645690917969, -0.025922056287527084, 0.0044490578584373, -0.05170980095863342, 0.02672792412340641, 0.02281946875154972, 0.0033426694571971893, 0.005170980002731085, 0.001036714413203299, 0.01202085055410862, 0.022537415847182274, 0.02272544987499714, 0.014505607075989246, 0.008783950470387936, 0.02637871541082859, -0.017635056748986244, -0.020845094695687294, -0.04072314873337746, -0.0015227529220283031, 0.04152901470661163, -0.016641154885292053, -0.003629759419709444, 0.009314480237662792, -0.017688781023025513, -0.05522875487804413, 0.008475035429000854, 0.003703630529344082, 0.014948833733797073, -0.0019743742886930704, -0.02558627910912037, 0.00328

In [ ]:
import faiss
import numpy as np
import pickle

# Criar matriz com os embeddings
X = np.array([item["embedding"] for item in embeddings_list]).astype("float32")

for i, item in enumerate(embeddings_list):
    embeddings_list[i]["embedding"] = X[i]

# FAISS usa índices, podemos escolher L2 por padrão
index = faiss.IndexFlatL2(X.shape[1])
index.add(X)

# Salvar FAISS index
faiss.write_index(index, "embeddings.index")

# Salvar metadados separados
with open("metadata.pkl", "wb") as f:
    pickle.dump(embeddings_list, f)

print("✅ Embeddings e metadados salvos!")

✅ Embeddings e metadados salvos!


In [14]:
avaliacoes = [
    {"query": "Qual contrato tem a empresa com sede na Rua Alfred Charvet", "relevante": [0]},  # índices dos chunks relevantes
    {"query": "ANDREY BARRETO MOLINARI COMERCIO", "relevante": [32, 50]},
    {"query": "Qual empresa é representada pelo sócio administrador Jhonny Cézar de Jesus Falavinhacontrato", "relevante": [0]}, 
    {"query": "Jhonny Cézar de Jesus Falavinhacontrato", "relevante": [0, 30, 53]},
    {"query": "76.443.340/0001-59", "relevante": [52]}
]  

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

import numpy as np

def gerar_embedding(texto):
     
    resp = client.embeddings.create(
        model="text-embedding-3-large",
        input=texto
    )
    return np.array(resp.data[0].embedding, dtype="float32")
